In [1]:
from test import *

BGM_lists = get_BGM_lists(YOUTUBE_CHANNEL)
print(BGM_lists)

{'BGM 2020 07': 'PLj63tGIo1sSq_E5c9ptp8xPBcS6tANq18', 'BGM 2020 08': 'PLj63tGIo1sSqZgEo3it5d7hnVo3lIgKRp', 'BGM 2020 09': 'PLj63tGIo1sSphQ5n5OEL2m_EzISuNwCpI', 'BGM 2020 10': 'PLj63tGIo1sSr17V1Rme01a18KXvHDOyRk', 'BGM 2020 11': 'PLj63tGIo1sSrymhqcZjGN5RpkXcx2-Qse', 'BGM 2020 12': 'PLj63tGIo1sSpUZJeAAAl30dsUmyJQvQ7J', 'BGM 2021 01': 'PLj63tGIo1sSo9GtiWnSNxr5PNCsRSgqFX', 'BGM 2021 02': 'PLj63tGIo1sSq9NXV8qpkzQaSFLXxK9Pji', 'BGM 2021 03': 'PLj63tGIo1sSrKSDvgeRhP3WJa7yszI247', 'BGM 2021 04': 'PLj63tGIo1sSqmoXWRh96aMgulu-VR0qzr', 'BGM 2021 05': 'PLj63tGIo1sSqYOG0UIx6G5BbJmDKl1TXx', 'BGM 2021 06': 'PLj63tGIo1sSqievrWIwa7k5VvcGMCatld', 'BGM 2021 07': 'PLj63tGIo1sSpDwSJxMGSED_gA1KVr8PdT', 'BGM 2021 08': 'PLj63tGIo1sSra3EYTtpean0kw9XOdJzd4', 'BGM 2021 09': 'PLj63tGIo1sSoJUOUTqnhpKryf87f1034o', 'BGM 2021 10': 'PLj63tGIo1sSrAijwosgz5PNrY6yQnqwkR', 'BGM 2021 11': 'PLj63tGIo1sSpmoVd1qrJMdL_qGWymbCw0', 'BGM 2021 12': 'PLj63tGIo1sSqrSt2TnyOgiJbtb1Tb0K0a'}


In [ ]:
p = multiprocessing.Pool(8)
start = time.time()
total_lists = []
works = p.imap_async(get_list_items, BGM_lists)
total_lists = works.get()
end = time.time()
print("working time: ", end - start)
print(total_lists)
p.close()
p.join()


![](youtube_playlist.png)

In [ ]:
import os
import pymysql
from dotenv import load_dotenv

load_dotenv()
DATABASE_HOST = os.getenv('DATABASE_HOST')
DATABASE_PORT = int(os.getenv('DATABASE_PORT'))
DATABASE_USER = os.getenv('DATABASE_USER')
DATABASE_PASSWORD = os.getenv('DATABASE_PASSWORD')
db = pymysql.connect(
        host=DATABASE_HOST,
        port=DATABASE_PORT,
        user=DATABASE_USER,
        passwd=DATABASE_PASSWORD,
        charset='utf8mb4'
)
cursor = db.cursor()

with open("schema.sql", 'r') as f:
    sqls = [x for x in f.read().split(";") if x != '']

for sql in sqls:
    cursor.execute(sql)
db.commit()
db.close()

In [ ]:
# SELECT name FROM done;
# INSERT INTO users (name) VALUES ('alskfl');
# SELECT name, uploader, href FROM songs
# INSERT INTO songs (name, uploader, href) VALUES ("{row[0]}", "{row[1]}", "{row[2]}")
# INSERT INTO done (name) VALUES ("{year}-{month}")